In [3]:
### vscode에서는 실행불가. jupyter를 이용하는 것을 권장 ###

# 라이브러리
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import glob
from natsort import natsorted
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

files = glob.glob('../Naver_Search_Amount/data/result/search_result_absolute*.csv') #최신파일 읽어오기
recent_file = natsorted(seq=files, reverse=True)[0]
df_anomaly = pd.read_csv(recent_file, encoding='cp949')
df_anomaly = df_anomaly[df_anomaly.columns[df_anomaly.apply(lambda x: len(x[x.notnull().values]) > 30).values == True]] # 값이 30개 이상인 항목만

def eval(df_anomaly):
    search_key = input('검색어를 입력해주세요: ')
    if search_key not in df_anomaly.columns:
        return print('해당 검색어는 이상감지 목표 검색어에 해당하지 않습니다. config 파일에 추가 후 재실행 해주십시오.')
    new_df = df_anomaly[['날짜', search_key]]
    new_df['날짜'] = new_df['날짜'].astype('datetime64[ns]')
    new_df = new_df.set_index('날짜')
    new_df_prophet = new_df.reset_index()[['날짜', new_df.columns[0]]].rename({'날짜':'ds', new_df.columns[0]:'y'}, axis='columns')
    model = Prophet() #seasonality_mode='multiplicative' 승법기법??
    model.add_country_holidays(country_name='KR')
    # 모델 학습(train을 분리하지 않은 이유는 분석목적이 '정확한 예측'이 아닌 '일반적인 수치'를 찾는 작업이기 때문-보고서 참조)
    model.fit(new_df_prophet)
    # train set 2022년 이후 날짜 생성
    year_2022 = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['ds']
    year_2022 = pd.DataFrame(year_2022, columns = ['ds'])
    year_2022['ds']= pd.to_datetime(year_2022['ds'])
    
    # 성능 평가(2022 그래프)
    forecast = model.predict(year_2022)
    model.plot(forecast)
    plt.xlim([datetime.date(2022, 1, 1), datetime.date(2022, 11, 1)])
    #plt.ylim([0, 30000])
    print('2022년 검색량 예측 결과입니다.')
    plt.show()

    # 성능 평가(2022 수치)
    # 여기에서는 MAE를 살펴본다.
    y_true = new_df_prophet[(new_df_prophet['ds'] >= '2022-01-01')]['y'].values
    y_pred = forecast['yhat'].values
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)*100
    print('MAE: %.3f' % mae)
    print('MAPE: %.3f' % mape)

    #전체 예측 성능(그래프)
    plt.plot(y_true, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.legend()
    #plt.ylim([0, 20000])
    plt.show()

eval(df_anomaly)

해당 검색어는 이상감지 목표 검색어에 해당하지 않습니다. config 파일에 추가 후 재실행 해주십시오.
